In [1]:
import pandas as pd
import psycopg2
from pymongo import MongoClient
import mongo_utilities
import os
from tqdm.notebook import tqdm
from datetime import datetime
from simpledbf import Dbf5

PyTables is not installed. No support for HDF output.


In [2]:
# Establish the connection with the database
db = mongo_utilities.connect()

Database connected successfully


In [ ]:
# Select the database
db = client['persistent']
# Create the collection(s)
collection = db['peru_exports']

In [ ]:
# Historical Load
# Get all the paths of the files to upload
folder = '../../data/temporal_landing/x/'
files = os.listdir('../../data/temporal_landing/x')
# Ingest the data into a collection
for file in tqdm(files):
    # Convert the data into a dataframe
    batch = Dbf5(folder+file, codec='latin-1')
    batch = batch.to_dataframe()
    # Add the loading date column
    batch['LOAD_DATE'] = datetime.today().strftime('%Y%m%d')
    # Convert the dataframe into a dict
    batch = batch.to_dict('records')
    try:
        collection.insert_many(batch)
    except Exception as e:
        print("An exception occurred ::", e)
        collection.drop()
        break

In [ ]:
mydoc = {"name": "John", "address": "Highway 37"}
x = collection.insert_one(mydoc)

In [ ]:
x.inserted_id

In [ ]:
# Establish the connection with the database
conn = db_utilities.connect()
cursor = conn.cursor()

In [ ]:
# Create the table(s)
cursor.execute(open("./create_tables.sql", "r").read())
conn.commit()

In [ ]:
# Historical Load
# Get all the paths of the files to upload
folder = '../../data/temporal_landing/x/'
files = os.listdir('../../data/temporal_landing/x')
# Populate the table in PostgreSQL
for file in tqdm(files):
    # Convert the data into a dataframe
    batch = Dbf5(folder+file, codec='latin-1')
    batch = batch.to_dataframe()
    # Add the loading date column
    batch['LOAD_DATE'] = datetime.today().strftime('%Y%m%d')
    batch.columns = batch.columns.str.lower()
    batch.to_sql('peru_exports', db_utilities.engine(), if_exists='append', index=False)

In [ ]:
cursor.execute('\dt')
results = cursor.fetchall()

In [ ]:
results

In [ ]:
conn.close()